# README
**Reminder to Windows and Mac OSX user:**
- auto-sklearn is not or partially supported on these systems, consider using Colab, VM, Docker.
- https://automl.github.io/auto-sklearn/master/installation.html#windows-osx-compatibility  
  
**General workflow for colab user**  
This notebook is organized in multiple steps, grouped in sections and subsections.
1. Set up  
    - install auto-sklearn
    - mount google drive
    - import packages
    - set paths for save/load
2. STEP A - Fetch datasets from OpenML (run once)
    - using sklearn API to fetch (multiple) datasets from OpenML
    - save each dataset as a pickle (.pkl), containing a dict with 'x', 'y' as keys and data as values, named as openml_datasetID.pkl
    - this step is intended to be run once only, afterwards user can load the data from pickle rather than fetch again which is time consuming. If you already have datasets in data folder, skip STEP A.
3. STEP B - Compute meta features (optional)
    - compute the meta features from pickle files, and write result to 'meta_features.pkl'
    - as meta info
    - to test whether the data loading works correctly
    - to test whether the saving result works correctly
    - can skip to STEP C
4. STEP C - Run experiment
    1. Set up
        - set up experiment by choosing budgets, datasets etc.
        - set up autosklearn classifier or modify settings
        - generates seed for reproducbility
        - check estimated time to run experiment, make sure you won't encounter session timeout
    2. Run
        - proceed to start experiment, a nested loop over datasets (outer) and seeds (inner) repeating the procedure 
            - train-test-split
            - train classifier on train set
           - evaluate balanced accuracy on test set
        - generate an unique timestamp id at the beginning
        - results are gathered in a 'res' dict with keys 'cls' and 'acc' containing the trained classifier and test accuracy
    3. Save (recommended)
        - save the 'res' dict to 'experiment_id.pkl' in results 
folder
        - storage warning: file size ~ 1.2GB for 190 trained classifiers + acc, make sure you have sufficient space, else do not save classifiers
        - avoid results loss in colab due to session timeout
        - analyze results in separate notebook to keep running experiment
        - (to do) saving results after each iteration of outer loop

**Particular instruction to colab user:**
- 'Run for colab' contains two cells, each with function to
    1.   install auto-sklearn on colab, then auto restart runtime. Run only once.
    2.   mount google drive for loading/saving datasets and results. In the examples in this notebook, dataset can be load from pickle and results can be dump to pickle, both stored at user-defined directories in google drive.

**Particular instruction to non-colab user:**
- Ignore 'Run for colab'
- Change the paths in 'Set paths' section


**File structure**  
The assumed file structure for the experiment.
  
    directory/
    -- Experiments.ipynb 
    -- Analyze_results.ipynb
    -- data/
    ---- openml_1234.pkl  
    -- results/
    ---- experiments_timestamp/ (note: current this subfolder is not functionable yet)
    ------ experiments_timestamp.pkl
    ------ fig_description_123.pkl

# INCLUDE TO SET UP

## Run for colab

In [ ]:
# For Colab, you need to install auto-sklearn every time
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    !pip install auto-sklearn # Downgrade scipy to 1.4.x
    #!pip install scipy # Upgrade scipy to 1.7.x

import os, signal
os.kill(os.getpid(), signal.SIGKILL) # Restart_runtime

     |████████████████████████████████| 6.3 MB 4.5 MB/s 
     |████████████████████████████████| 38.1 MB 1.3 MB/s 
     |████████████████████████████████| 22.3 MB 11.4 MB/s 
     |████████████████████████████████| 722 kB 26.0 MB/s 
     |████████████████████████████████| 4.2 MB 42.7 MB/s 
     |████████████████████████████████| 4.0 MB 39.8 MB/s 
     |████████████████████████████████| 208 kB 24.6 MB/s 
     |████████████████████████████████| 973 kB 47.3 MB/s 
     |████████████████████████████████| 132 kB 65.1 MB/s 
     |████████████████████████████████| 45 kB 3.3 MB/s 
  Created wheel for auto-sklearn: filename=auto_sklearn-0.14.2-py3-none-any.whl size=6586738 sha256=3e75c2248e3ba5bd48d30039d077a7f2141adc5f5b54b27b2fb778f26dabe67d
  Stored in directory: /root/.cache/pip/wheels/b6/ba/aa/ce7b8deb41019b9c1c6cd0b511d553350b7075ba1d3f0f545c
  Created wheel for pynisher: filename=pynisher-0.6.4-py3-none-any.whl size=7044 sha256=654c9eea1371ad0f6b8e97f8e7e50a8dc897c637dc65e6b5f9bfb08a8df0cb

In [ ]:
import sys
IN_COLAB = 'google.colab' in sys.modules

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Run for everyone

In [ ]:
# Dependencies
# Common
import os
import pickle
import sys
import time
from tqdm.autonotebook import tqdm
from datetime import datetime

import numpy as np
import pandas as pd

# Plot
import matplotlib.pyplot as plt
import seaborn as sns

# ML
import sklearn # Import sklearn before autosklearn, solve scipy version error
from sklearn.model_selection import train_test_split
import sklearn.datasets
from sklearn.metrics import accuracy_score, balanced_accuracy_score

# AML
import autosklearn
import autosklearn.classification
from autosklearn.metrics import balanced_accuracy, precision, recall

# Check machine
import multiprocessing
multiprocessing.cpu_count()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  import sys


2

## Set paths

In [ ]:
# Set up paths

# User insert folder to store pickles
# dataset name in format openml_xxx.pkl
datasets_folder = '/content/drive/MyDrive/Colab Notebooks/AML_experiments/all_data'
results_folder = '/content/drive/MyDrive/Colab Notebooks/AML_experiments/results'

In [ ]:
# Example of writing to google drive
#with open('/content/drive/My Drive/foo.txt', 'w') as f:
#  f.write('Hello Google Drive!')
#!cat /content/drive/My\ Drive/foo.txt

# List all classifiers in auto-sklearn
https://automl.github.io/auto-sklearn/master/examples/40_advanced/example_interpretable_models.html#sphx-glr-examples-40-advanced-example-interpretable-models-py

In [ ]:
from autosklearn.pipeline.components.classification import ClassifierChoice

for name in ClassifierChoice.get_components():
    print(name)

adaboost
bernoulli_nb
decision_tree
extra_trees
gaussian_nb
gradient_boosting
k_nearest_neighbors
lda
liblinear_svc
libsvm_svc
mlp
multinomial_nb
passive_aggressive
qda
random_forest
sgd


# WORKFLOW


## STEP A: Fetch dataset and store in pickle (run once)
sklearn fetch from openml https://scikit-learn.org/stable/datasets/loading_other_datasets.html

Error: dataset 41147 gave internal server error

In [ ]:
dataset_ids = [54, 41165, 41161, 41159, 41163,
               41142, 1468, 41164, 40996, 
               1111, 12, 41166, 41138,
               41143, 1486, 
               #41147,  # exclude due to internal server error
               41167, 
               41168, 1596, 41150, 40668,
               3, 23512, 41169, 1067,
               23517, 41146, 40984,
               1461, 1590,
               4135, 40685, 1169,
               41027] # probably incorrect

In [ ]:
# # Fetch OpenML datasets

# # All dataset ids are to be downloaded 
# # except 41147 that returned error
# # Dataset ids extracted from  Appendix D, Table 20 from the autosklearn2.0 paper
# dataset_ids = [41165, 41161, 41159, 41163,
#                41142, 1468, 41164, 40996, 
#                1111, 12, 41166, 41138,
#                41143, 1486, 
#                #41147,  # exclude due to internal server error
#                41167, 
#                41168, 1596, 41150, 40668,
#                3, 23512, 41169, 1067,
#                23517, 31, 41146, 40984,
#                54, 1461, 40981, 1590,
#                4135, 40685, 1169, 40975,
#                41027, 1489, 1464]

# # User insert folder to store pickles
# folder_path = datasets_folder

# # Fetch and save each dataset
# for dataset_id in dataset_ids:
#     print(f'Fetching data: {dataset_id}')
#     # Using fetch function from sklearn
#     x, y = sklearn.datasets.fetch_openml(data_id=dataset_id, 
#                                          return_X_y=True, 
#                                          as_frame=True) # Return dataframe
#     # Store as dict
#     data = {'x': x, 'y': y}
#     # Dump dict to pickle
#     file_to_dump = data

#     fname = os.path.join(folder_path, 'openml_' + str(dataset_id))
#     file_write = open(fname, 'wb')
#     # "wb" mode opens the file in binary format for writing
#     pickle.dump(file_to_dump, file_write)
#     file_write.close()

## STEP B: Test data loading and compute meta features of test dataset, test result saving 
- Compute meta features of all datasets.
- Also serves as checking whether data can be successfully loaded
- Takes < 1 min to run

In [ ]:
meta_features = {}

folder_path = datasets_folder
all_dataset_names = os.listdir(folder_path)

print(f'Total # datasets = {len(all_dataset_names)}')

for dataset_name in all_dataset_names:

    # Load data
    fname = os.path.join(folder_path, dataset_name)
    print(fname)
    if not fname.endswith(".pkl"):
      continue
    file_read = open(fname, "rb")
    data = pickle.load(file_read) # Load pickle to data
    file_read.close()

    # dataset name in format openml_xxx.pkl
    # extract xxx as key
    dataset_id = dataset_name.replace('_', '.').split('.')[1]

    x, y = data['x'], data['y']
    print(f'\nCompute meta features of test dataset: {dataset_id}')
    print(f'meta feature: # instance {x.shape[0]}, # feature {x.shape[1]}')
    meta_features[str(dataset_id)] = {'n_instance': x.shape[0], # Row
                                      'n_feature': x.shape[1], # Col
                                      'n_class': len(np.unique(y))}    

Total # datasets = 38
/content/drive/MyDrive/Colab Notebooks/AML_experiments/all_data/openml_40981.pkl

Compute meta features of test dataset: 40981
meta feature: # instance 690, # feature 14
/content/drive/MyDrive/Colab Notebooks/AML_experiments/all_data/openml_40975.pkl

Compute meta features of test dataset: 40975
meta feature: # instance 1728, # feature 6
/content/drive/MyDrive/Colab Notebooks/AML_experiments/all_data/openml_1464.pkl

Compute meta features of test dataset: 1464
meta feature: # instance 748, # feature 4
/content/drive/MyDrive/Colab Notebooks/AML_experiments/all_data/openml_1489.pkl

Compute meta features of test dataset: 1489
meta feature: # instance 5404, # feature 5
/content/drive/MyDrive/Colab Notebooks/AML_experiments/all_data/openml_31.pkl

Compute meta features of test dataset: 31
meta feature: # instance 1000, # feature 20
/content/drive/MyDrive/Colab Notebooks/AML_experiments/all_data/openml_54.pkl

Compute meta features of test dataset: 54
meta feature: # i

In [ ]:
# Save meta features
folder_path = results_folder

# Dump dict to pickle
file_to_dump = meta_features
fname = os.path.join(folder_path, 'openml_meta_features.pkl')
file_write = open(fname, 'wb')
# "wb" mode opens the file in binary format for writing
pickle.dump(file_to_dump, file_write)
file_write.close()

## STEP C: Run experiments

### Experiment 1 part 1

#### Set up experiment
- exp id =  'experiment_20211202-220329.pkl'
- time budget = 60s (1 min)
- memory budget = 3072MB
- datasets = 38 datasets

In [ ]:
# Dataset

# User insert folder to store pickles
folder_path = datasets_folder
# Datasets D_new to evaluate
# All datasets located in the datasets_folder directory set up by user
all_dataset_names = os.listdir(folder_path)

# Auto-sklearn hyperparameters
# Time limit in seconds for the search of appropriate models.
time_left_for_this_task = 60 # Time budget
#per_run_time_limit = int(time_left_for_this_task / 10) # Default is budget/10. Longer the more training time.
# Memory limit in MB for the machine learning algorithm.
memory_limit = 3072 # Memory budget

# Repeat evaluation with different seeds
repeat = 5 

# Set random seed generator
rng = np.random.default_rng(int(123456)) # Fix seed for reproducibility
seeds = rng.integers(0, 2**32-1, repeat) # Max seed limited by autoskl

seed_split = 12345 # Fix seed for reproducibility on train test split


# Check estimated experiment duration before proceed
print(f'Est. time required: {repeat * time_left_for_this_task * len(all_dataset_names) / 60} min')
print(f'seeds: {seeds}')

Est. time required: 30.0 min
seeds: [ 422765254 2733805738 3503586592 1652753514 1605617222]


#### Start experiment

In [ ]:
# Run experiment

# Set unique identifier
timestamp = datetime.now().strftime('%Y%m%d-%H%M%S')
exp_id = "experiment_" + timestamp

number_of_random_search = [i for i in range(0, 100)]

# Tqdm bars
outter_bar = tqdm(all_dataset_names, desc='dataset', leave=False)
inner_bar = tqdm(number_of_random_search, desc='Random search', leave=False)

# Store result
res = {}

# Start experiment
print(f'\nStart running experiment id: {exp_id}')
start = time.perf_counter()

for dataset_name in all_dataset_names:
    print(dataset_name)
    # if(dataset_name not in dataset_ids):
    #   print("continue")
    #   continue
    outter_bar.update(1) # tqdm progress bar + 1

    # Load data
    fname = os.path.join(folder_path, dataset_name)
    file_read = open(fname, "rb")
    data = pickle.load(file_read) # Load pickle to data
    file_read.close()

    # dataset name in format openml_xxx.pkl
    # extract xxx as key
    dataset_id = dataset_name.replace('_', '.').split('.')[1]

    x, y = data['x'], data['y']
    print(f'\nStart training on dataset: {dataset_id}')
    print(f'meta feature: # instance = {x.shape[0]}, # feature = {x.shape[1]}')

    # Split data
    x_train, x_test, y_train, y_test = train_test_split(
        x, y, test_size=0.2, random_state=seed_split)
    seed = 18
    res_cls = []
    res_acc = []
    for i in number_of_random_search:

        inner_bar.update(1) # tqdm progress bar + 1

        # Train
        classifier = np.random.choice(["random_forest","adaboost","bernoulli_nb","decision_tree","extra_trees","gradient_boosting","k_nearest_neighbors","liblinear_svc","libsvm_svc","mlp","multinomial_nb","passive_aggressive","qda","random_forest","sgd"])
        ensembleSize = np.random.randint(10, 60)
        ensembleNBest = ensembleSize
        maxModelsOnDisc = np.random.randint(1, ensembleSize)
        resamplingStrategy = np.random.choice(["holdout", "holdout-iterative-fit", "cv", "cv-iterative-fit"])
        print(f"Ensemble size: {ensembleSize}")
        cls = autosklearn.classification.AutoSklearnClassifier(
            time_left_for_this_task=time_left_for_this_task,
            per_run_time_limit=10,
            memory_limit=None,
            seed=int(seed),
            ensemble_size = ensembleSize,
            ensemble_nbest = ensembleNBest,
            max_models_on_disc = maxModelsOnDisc,
            resampling_strategy = resamplingStrategy,
            include = {
                'classifier': [classifier]
            },
            exclude = None
        )
        # Feurer et al.(2021): single metric for binary clf, multiclass clf and unbalanced datasets

        # Train
        cls.fit(x_train, y_train, dataset_name=dataset_id)
        
        # Test on trained classfier
        predictions = cls.predict(x_test)
        test_acc = balanced_accuracy_score(y_test, predictions)

        # Store
        res_cls.append(cls)
        res_acc.append(test_acc)
    
    inner_bar.reset() # reset the seed progress bar for next dataset

    res[dataset_id] = {'cls': res_cls, 'acc': res_acc}

    # Save result and trained model to pickle for persitence

    # Save
    file_dir = results_folder
    file_path = os.path.join(file_dir, exp_id)
    file_write = open(file_path, "wb")
    # "wb" mode opens the file in binary format for writing
    pickle.dump(res, file_write)
    file_write.close()


# Finish experiment
end = time.perf_counter()

print(f'\nExperiment completed in: {(end-start)/60:.1f} min')

dataset:   0%|          | 0/6 [00:00<?, ?it/s]

Random search:   0%|          | 0/100 [00:00<?, ?it/s]


Start running experiment id: experiment_20211215-150746
openml_41167.pkl

Start training on dataset: 41167
meta feature: # instance = 416188, # feature = 60
Ensemble size: 59
[WARNING] [2021-12-15 15:07:59,367:Client-AutoMLSMBO(18)::41167] Configuration 102 not found
[WARNING] [2021-12-15 15:07:59,369:Client-AutoMLSMBO(18)::41167] Configuration 291 not found
[WARNING] [2021-12-15 15:07:59,369:Client-AutoMLSMBO(18)::41167] Configuration 243 not found
[WARNING] [2021-12-15 15:07:59,369:Client-AutoMLSMBO(18)::41167] Configuration 134 not found
[WARNING] [2021-12-15 15:07:59,369:Client-AutoMLSMBO(18)::41167] Configuration 448 not found
[WARNING] [2021-12-15 15:07:59,369:Client-AutoMLSMBO(18)::41167] Configuration 543 not found
[WARNING] [2021-12-15 15:07:59,369:Client-AutoMLSMBO(18)::41167] Configuration 298 not found
[WARNING] [2021-12-15 15:07:59,369:Client-AutoMLSMBO(18)::41167] Configuration 171 not found
[WARNING] [2021-12-15 15:07:59,369:Client-AutoMLSMBO(18)::41167] Configuration 5

ValueError: ignored

In [ ]:
while True:
  pass

## Experiment 1
Used datasets: 40981, 40975, 1464, 1489, 31
## Experiment 2
Used datasets: 4135, 1416, 1067, 1486, 54, 41143, 40984, 41146, 3
## Experiment 3
Used datasets: 41027, 23512, 1590, 12
## Experiment 4 of 5?
Used datasets: 41169, 41166, 40668
## Experiment 6
Used datasets: 40685, 1169, 41138, 41150, 1468
## Experiment 7
Used datasets: 1596, 41163

#### Saving trained classifiers, test accuracy to pickle
WARNING: file size may be large
1.12GB

In [ ]:
# Save result and trained model to pickle for persitence

# Save
file_dir = results_folder
file_path = os.path.join(file_dir, exp_id)
file_write = open(file_path, "wb")
# "wb" mode opens the file in binary format for writing
pickle.dump(res, file_write)
file_write.close()

#### Logs

All dataset completed without warnings except for:

    Start training on dataset: 1111
    meta feature: # instance 50000, # feature 230
    /usr/local/lib/python3.7/dist-packages/autosklearn/data/feature_validator.py:209: SettingWithCopyWarning: 
    A value is trying to be set on a copy of a slice from a DataFrame.
    Try using .loc[row_indexer,col_indexer] = value instead


#### Process results (suggest to be done separately after saving res file)

In [ ]:
pd.DataFrame({k: {'mean_acc': np.mean(v['acc']), 'sd_acc': np.std(v['acc'])} for k,v in res.items()}).T